In [1]:
#preprocessing.ipynb
import cv2
import os
import matplotlib.pyplot as plt


def preprocess_image(image_path):
    print(f"Reading image from: {image_path}")  # Debugging statement
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")

    # Convert the image to YCrCb color space
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Split the YCrCb image to different channels
    y, cr, cb = cv2.split(ycrcb_image)
    
    # Apply histogram equalization to the Y channel for enhancement
    y_enhanced = cv2.equalizeHist(y)
    
    # Merge the enhanced Y channel back with Cr and Cb channels
    ycrcb_enhanced = cv2.merge([y_enhanced, cr, cb])
    
    # Convert the enhanced YCrCb image back to BGR color space
    enhanced_image = cv2.cvtColor(ycrcb_enhanced, cv2.COLOR_YCrCb2BGR)
    
    # Convert the enhanced image to grayscale
    gray_image = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
    
    # Apply binary thresholding for segmentation
    _, binary_image = cv2.threshold(gray_image, 66, 255, cv2.THRESH_BINARY)
    
    # Apply the mask to the original image to segment the leaf
    masked_image = cv2.bitwise_and(image, image, mask=binary_image)
    
    # Perform Sobel edge detection
    sobelx = cv2.Sobel(binary_image, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(binary_image, cv2.CV_64F, 0, 1, ksize=5)
    sobel_combined = cv2.magnitude(sobelx, sobely)
    
    # Perform Canny edge detection
    canny_edges = cv2.Canny(binary_image, 100, 200)
    
    return masked_image, enhanced_image, gray_image, binary_image, sobel_combined, canny_edges


